In [1]:
from demucs import pretrained
import torch
from demucs.apply import apply_model
import timeit
import gc

In [2]:
models_to_test = ['mdx', 'htdemucs', 'htdemucs_ft', 'htdemucs_6s', 'hdemucs_mmi', 'mdx_extra']

## Env
### Hardware
- 2022 MacBook Pro
- Apple M2 Chip
- 24 GB RAM
### Software
- Python 3.10.11
- torch-2.1.0.dev20230601
-  torchaudio-2.1.0.dev20230601

In [5]:
def time_model(model_name, device, grad=False, n=3):
    model = pretrained.get_model(model_name)
    x = torch.randn(1, 2, 44100 * 30).to(device=device)  # 30 seconds of white noise for the test
    
    if grad:
        time = timeit.timeit(lambda: apply_model(model, x, device=device), number=n)
    else:
        with torch.no_grad():
            time = timeit.timeit(lambda: apply_model(model, x, device=device), number=n)

    del model
    del x
    gc.collect()

    time /= float(n)
    return time

In [6]:
n = 1

for model_name in models_to_test:
    for grad in [True, False]:
        print(f"Testing {model_name} {'with' if grad else 'without'} gradients")
        
        cpu_time = time_model(model_name, 'cpu', grad=grad, n=n)
        mps_time = time_model(model_name, 'mps', grad=grad, n=n)

        print(f"CPU time: {cpu_time:.2f}s, MPS time: {mps_time:.2f}s")
        print(f"Speedup: {cpu_time / mps_time:.2f}x")
        print()



Testing mdx with gradients
CPU time: 21.26s, MPS time: 11.79s
Speedup: 1.80x

Testing mdx without gradients
CPU time: 21.26s, MPS time: 13.40s
Speedup: 1.59x

Testing htdemucs with gradients
CPU time: 17.18s, MPS time: 6.94s
Speedup: 2.48x

Testing htdemucs without gradients
CPU time: 17.19s, MPS time: 4.23s
Speedup: 4.06x

Testing htdemucs_ft with gradients
CPU time: 68.47s, MPS time: 16.78s
Speedup: 4.08x

Testing htdemucs_ft without gradients
CPU time: 67.96s, MPS time: 16.69s
Speedup: 4.07x

Testing htdemucs_6s with gradients
CPU time: 17.09s, MPS time: 4.86s
Speedup: 3.52x

Testing htdemucs_6s without gradients
CPU time: 16.52s, MPS time: 4.28s
Speedup: 3.86x

Testing hdemucs_mmi with gradients
CPU time: 6.60s, MPS time: 5.83s
Speedup: 1.13x

Testing hdemucs_mmi without gradients
CPU time: 6.60s, MPS time: 3.85s
Speedup: 1.71x

Testing mdx_extra with gradients
CPU time: 26.35s, MPS time: 28.73s
Speedup: 0.92x

Testing mdx_extra without gradients
CPU time: 26.07s, MPS time: 22.78s
